# Getting started With Langchain And Open Al
In this quickstart we'll see how to:
- Get setup with LangChain, LangSmith and LangServe
- Use the most basic and common components of LangChain: prompt templates, models, and - output parsers.
- Build a simple application with LangChain
- Trace your application with LangSmith
- Serve your application with LangServe

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['LaNGCHAIN_API_KEY'] = os.getenv('LaNGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRACING_V2']="true"
os.environ['LANGCHAIN_PROJECT']=os.getenv('LANGCHAIN_PROJECT')

In [2]:
from langchain_community.document_loaders import WebBaseLoader
loader=WebBaseLoader(web_path="https://blogs.nvidia.com/blog/what-is-agentic-ai/")
docs=loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
docs[0].page_content

"\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nWhat Is Agentic AI?\xa0 | NVIDIA Blog\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\t\t\t\t\t\t\t\tArtificial Intelligence Computing Leadership from NVIDIA\t\t\t\t\t\t\t\n\n\n\n\n\n\n\n\n\n\n\nSearch for:\n\n\n\n\n\n \n\n\n\n\nToggle Search\n\n\n\n\n\n\n\n\n\n\n\n\nHome\nAI\nData Center\nDriving\nGaming\nPro Graphics\nRobotics\nHealthcare\nStartups\nAI Podcast\nNVIDIA Life\n \n\n\n\n\n\n\n\n\n\n\n\nWhat Is Agentic AI?\xa0\n\n\n\t\t\t\tAgentic AI uses sophisticated reasoning and iterative planning to autonomously solve complex, multi-step problems.\t\t\t\n\n\nOctober 22, 2024 by Erik Pounds \n\n\n\n\n\n\n\n\n\n\n\n\t\t\t\t\tShare\t\t\t\n\n\n \n\n\n\n\n \n \n \nEmail0 \n\n\n  Editor’s note: The name of NIM Agent Blueprints was changed to NVIDIA\xa0Blueprints in October 2024. All references to the name have

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter=RecursiveCharacterTextSplitter(chunk_size=200,chunk_overlap=20)
splitter

In [7]:
split_docs=splitter.split_documents(docs)

In [9]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
embeddings = OpenAIEmbeddings()
vs=FAISS.from_documents(split_docs, embeddings)


In [10]:
# documet chain and retriever chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """Answer the following questions only based on context provided.
    <context>
    {context}
    </context>
    question: {input}
    """
)


In [11]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(
    model='gpt-3.5-turbo',
)

In [12]:
# document chain : To give context to the prompt and get response from the llm
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)

In [13]:
print(document_chain)

bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='Answer the following questions only based on context provided.\n    <context>\n    {context}\n    </context>\n    question: {input}\n    '), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x00000217091D6F90>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000217091D7620>, root_client=<openai.OpenAI object at 0x000002170B108410>, root_async_client=<openai.AsyncOpenAI object at 0x000002170B1091D0>, model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser() kwargs

In [15]:
# retriver chain to get the response from the vector store
from langchain.chains import create_retrieval_chain
retriever=vs.as_retriever(
    search_type="similarity",
    search_kwargs={"k":3}
)

retriever_chain=create_retrieval_chain(retriever,document_chain)

In [16]:
# get response from the llm
result=retriever_chain.invoke({
    "input":"what is the changes in the new version of langchain?"
})

In [22]:
# get keys of dictionary
print(result.keys())

dict_keys(['input', 'context', 'answer'])


In [24]:
print(result['answer'])

The changes in the new version of Langchain involve incorporating a large language model as the orchestrator for understanding tasks, generating solutions, and coordinating specialized models for specific functions.
